## Import Libaries

In [1]:
from io import StringIO
import pandas as pd
import json

In [2]:
import plotly

In [3]:
!pip install --upgrade certifi

In [4]:
dmtool_userid = '1'
dmtool_apikey = '20CA2CF4_3DE3_11EF_98A4_D60CFA69E605'

In [5]:
from dmtools_api_package.dmtools_api_module import APIServer

In [6]:
MyAPIServer = APIServer(dmtool_userid,dmtool_apikey)

In [7]:
from dmtools_api_package.dmtools_api_module import DataExamples

In [8]:
from dmtools_api_package.dmtools_api_module import Data_about_example
dae = Data_about_example()
#dae.json
response = MyAPIServer.create_and_return_a_data('', dae.json)

In [32]:
type(response)

dict

In [10]:
de = DataExamples()

In [11]:
de.MakeDMTool(1000,1)

In [12]:
de.dmtdf.head(5)

,x,y,data_id,trace_id,trace_name,row_id
0,30,0,1000,1,d,0
1,65,5,1000,1,d,1
2,90,25,1000,1,d,2
3,100,50,1000,1,d,3
4,90,75,1000,1,d,4


In [13]:
import plotly.graph_objects as go
#fig = go.Figure( go.Scatter(x=[1,2,3], y=[1,3,2] ) )
#fig.show()

## XY For Test Data

In [33]:
all_letters = de.dmtdf.copy()
#all_letters = all_letters[['data_label', 'trace_name','data_id', 'row_id','trace_id','x', 'y']]
all_letters['y'] = all_letters['y'] * 1.123456789012e-45
all_letters.head(5)

,x,y,data_id,trace_id,trace_name,row_id
0,30,0.000000e+00,1000,1,d,0
1,65,5.617284e-45,1000,1,d,1
2,90,2.808642e-44,1000,1,d,2
3,100,5.617284e-44,1000,1,d,3
4,90,8.425926e-44,1000,1,d,4


In [15]:
all_letters['data_id'].unique()

array([1000])

In [16]:
# Specify field column names
fieldcols = ['trace_id','x','y']

# Build a dict for each group as a Series named `fields`
'''
res = (all_letters.groupby(['data_id'])
 .apply(lambda s: [{'trace_id' : traceid,
                    'x': valuex,
                    'y': valuey}
                   for field in fieldcols
                   for traceid in s['trace_id'].values
                   for valuex in s['x'].values
                   for valuey in s['y'].values]
                  )
).rename('data_values')
'''

## working
# Group by 'data_id' and build the list of dictionaries
res = (all_letters.groupby('data_id')
       .apply(lambda s: s.apply(lambda row: {'trace_id': row['trace_id'],'row_id': row['row_id'], 'x': row['x'], 'y': row['y']}, axis=1).tolist())
       .rename('data_values')
       .reset_index())


''' the following changes the order of the output
res_lol = (all_letters.groupby('data_id')
       .apply(lambda s: [{row['trace_id'], row['x'], row['y']} for _, row in s.iterrows()])
       .rename('data_values')
       .reset_index())
'''

'''
res = (all_letters.groupby('data_id')
       .apply(lambda s: {str(row['trace_id']): {'x': str(row['x']), 'y': str(row['y'])} for _, row in s.iterrows()}, axis=1).tolist()
       .rename('data_values')
       .reset_index())

'''
'''

res = (all_letters.groupby('data_id')
       .apply(lambda s: {row['trace_id']: {'x': row['x'], 'y': row['y']} for _, row in s.iterrows()})
       .rename('data_values')
       .reset_index())
'''
# Convert to JSON
json_result = res.to_json(orient='records')


# Convert Series to DataFrame and then to_json
#res = res.reset_index().to_json(orient='records')

json_result

/tmp/ipykernel_9924/3463747819.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda s: s.apply(lambda row: {'trace_id': row['trace_id'],'row_id': row['row_id'], 'x': row['x'], 'y': row['y']}, axis=1).tolist())


'[{"data_id":1000,"data_values":[{"trace_id":1,"row_id":0,"x":30,"y":0.0},{"trace_id":1,"row_id":1,"x":65,"y":5.617283945e-45},{"trace_id":1,"row_id":2,"x":90,"y":2.808641973e-44},{"trace_id":1,"row_id":3,"x":100,"y":5.617283945e-44},{"trace_id":1,"row_id":4,"x":90,"y":8.425925918e-44},{"trace_id":1,"row_id":5,"x":65,"y":1.06728395e-43},{"trace_id":1,"row_id":6,"x":30,"y":1.123456789e-43},{"trace_id":1,"row_id":7,"x":60,"y":9.549382707e-44},{"trace_id":1,"row_id":8,"x":75,"y":8.425925918e-44},{"trace_id":1,"row_id":9,"x":80,"y":5.617283945e-44},{"trace_id":1,"row_id":10,"x":75,"y":3.932098762e-44},{"trace_id":1,"row_id":11,"x":60,"y":1.685185184e-44},{"trace_id":1,"row_id":12,"x":30,"y":0.0},{"trace_id":1,"row_id":13,"x":10,"y":0.0},{"trace_id":1,"row_id":14,"x":10,"y":1.123456789e-43},{"trace_id":1,"row_id":15,"x":30,"y":1.123456789e-43},{"trace_id":1,"row_id":16,"x":30,"y":0.0},{"trace_id":2,"row_id":17,"x":170,"y":5.617283945e-44},{"trace_id":2,"row_id":18,"x":190,"y":1.123456789e-4

In [17]:
#json_df = pd.read_json(json_result, orient='records')
#json_df
data = json.loads(json_result)
#df = pd.json_normalize(data['data_values'])
#df = pd.DataFrame(data=data)
data[0]['data_id']
len_data = len(data)

data_df = pd.DataFrame(data=[])

for i in range(0,len_data):
    data_id = data[i]['data_id']
    data_values = data[i]['data_values']
    new_df = pd.DataFrame(data=data_values)
    new_df['data_id'] = data_id
    data_df = pd.concat([data_df,new_df])
data_df.head(5)

,trace_id,row_id,x,y,data_id
0,1,0,30,0.000000e+00,1000
1,1,1,65,5.617284e-45,1000
2,1,2,90,2.808642e-44,1000
3,1,3,100,5.617284e-44,1000
4,1,4,90,8.425926e-44,1000


In [18]:
chart_data = data_df[data_df['data_id']==1000]
chart_data.head(5)

,trace_id,row_id,x,y,data_id
0,1,0,30,0.000000e+00,1000
1,1,1,65,5.617284e-45,1000
2,1,2,90,2.808642e-44,1000
3,1,3,100,5.617284e-44,1000
4,1,4,90,8.425926e-44,1000


In [19]:
def create_letter_color(letter_in):
    color_out = 'blue'
    if letter_in == 'd':
        color_out = 'red'
    elif letter_in == 'm':
        color_out = 'orange'
    elif letter_in == 't':
        color_out = 'yellow'
    elif letter_in == 'o1':
        color_out = 'green'
    elif letter_in == 'o2':
        color_out = 'blue'
    elif letter_in == 'l':
        color_out = 'indigo'
    return color_out

def create_trace_color(number_in):
    color_out = 'blue'
    if number_in == 1:
        color_out = 'red'
    elif number_in == 2:
        color_out = 'orange'
    elif number_in == 3:
        color_out = 'yellow'
    elif number_in == 4:
        color_out = 'green'
    elif number_in == 5:
        color_out = 'blue'
    elif number_in == 6:
        color_out = 'indigo'
    return color_out

#dmdf['color'] = dmdf.apply(lambda row: create_letter_color(row['trace_name']),axis=1)
trace_ids = data_df['trace_id'].unique().tolist()
trace_ids

[1, 2, 3, 4, 5, 6]

In [20]:
trace_id = chart_data['trace_id'].unique()
trace_id

array([1, 2, 3, 4, 5, 6])

In [21]:
fig = go.Figure()

for t in trace_ids:
      trace_data = chart_data[chart_data['trace_id']==t]
      trace_id = t
      trace_color = create_trace_color(t)
      print(trace_id)
      fig.add_trace(go.Scatter(
          x=trace_data['x'], y=trace_data['y'],
          name=str(t),
          mode='lines',
          line_color=trace_color
      ))
#fig.update_yaxes(type="linear")
fig.update_yaxes(type="log")
#fig.show(renderer="svg")
plotly.offline.plot(fig, filename = 'plot.html')

1
2
3
4
5
6


'plot.html'

In [22]:
# 'Experiment DMTOOL'
index_id = 1
data_about = pd.DataFrame(data=[], columns=[])
data_about.loc[index_id,'data_subject'] = 'data_about'
##data_about.loc[index_id,'limit_id'] = 1001
##data_about.loc[index_id,'data_id'] = 1001
data_about.loc[index_id,'plot_id'] = 1001
data_about.loc[index_id,'series_id'] = 1001
data_about.loc[index_id,'data_label'] = 'dmtool'
data_about.loc[index_id,'data_reference'] = 'dmtool'
data_about.loc[index_id,'data_comment'] = 'Experiment DMTOOL'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment DMTOOL'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0

#just_this = data_about.loc[index_id]

data_about_json = data_about.to_json(orient="records")
data_about_json
#value_var_list = list(limits_metadata_df.columns)
#value_var_list.remove('limit_id')

'[{"data_subject":"data_about","plot_id":1001.0,"series_id":1001.0,"data_label":"dmtool","data_reference":"dmtool","data_comment":"Experiment DMTOOL","x_units":"GeV","y_units":"cm^2","x_rescale":"1","y_rescale":"1","year":"2023","experiment":"Experiment DMTOOL","spin_dependency":"SD","result_type":"Th","official":"1","open":"1","greatest_hit":"0","public":"1","created_at":"2024-07-08 11:07:00","updated_at":"2024-07-08 11:07:00","archived_at":"2024-07-08 11:07:00","creator_id":0.0,"rating":0.0,"date_of_announcement":"2024-07-08 11:07:00","date_official":"2024-07-08 11:07:00","date_of_run_start":"2024-07-08 11:07:00","date_of_run_end":"2024-07-08 11:07:00","default_color":"black","default_style":"dotted"}]'

In [23]:
#data_about

In [24]:
# 'Experiment D'
index_id = 2

data_about = pd.DataFrame(data=[], columns=[])
data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1002
data_about.loc[index_id,'data_id'] = 1002
data_about.loc[index_id,'plot_id'] = 1002
data_about.loc[index_id,'series_id'] = 1002
data_about.loc[index_id,'data_label'] = 'd'
data_about.loc[index_id,'data_reference'] = 'd'
data_about.loc[index_id,'data_comment'] = 'Experiment D'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment D'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0


#value_var_list = list(limits_metadata_df.columns)
#value_var_list.remove('limit_id')

In [25]:
# 'Experiment M'
index_id = 3

data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1003
data_about.loc[index_id,'data_id'] = 1003
data_about.loc[index_id,'plot_id'] = 1003
data_about.loc[index_id,'series_id'] = 1003
data_about.loc[index_id,'data_label'] = 'm'
data_about.loc[index_id,'data_reference'] = 'm'
data_about.loc[index_id,'data_comment'] = 'Experiment M'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment M'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0


#value_var_list = list(limits_metadata_df.columns)
#value_var_list.remove('limit_id')

In [26]:
# 'Experiment T'
index_id = 4

data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1004
data_about.loc[index_id,'data_id'] = 1004
data_about.loc[index_id,'plot_id'] = 1004
data_about.loc[index_id,'series_id'] = 1004
data_about.loc[index_id,'data_label'] = 't'
data_about.loc[index_id,'data_reference'] = 't'
data_about.loc[index_id,'data_comment'] = 'Experiment T'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment T'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0

In [27]:
# 'Experiment O1'
index_id = 5

data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1005
data_about.loc[index_id,'data_id'] = 1005
data_about.loc[index_id,'plot_id'] = 1005
data_about.loc[index_id,'series_id'] = 1005
data_about.loc[index_id,'data_label'] = 'o1'
data_about.loc[index_id,'data_reference'] = 'o1'
data_about.loc[index_id,'data_comment'] = 'Experiment O1'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment O1'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0

In [28]:
# 'Experiment O2'
index_id = 6

data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1006
data_about.loc[index_id,'data_id'] = 1006
data_about.loc[index_id,'plot_id'] = 1006
data_about.loc[index_id,'series_id'] = 1006
data_about.loc[index_id,'data_label'] = 'o2'
data_about.loc[index_id,'data_reference'] = 'o2'
data_about.loc[index_id,'data_comment'] = 'Experiment O2'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment O2'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0

In [29]:
# 'Experiment L'
index_id = 7

data_about.loc[index_id,'data_subject'] = 'data_about'
data_about.loc[index_id,'limit_id'] = 1007
data_about.loc[index_id,'data_id'] = 1007
data_about.loc[index_id,'plot_id'] = 1007
data_about.loc[index_id,'series_id'] = 1007
data_about.loc[index_id,'data_label'] = 'l'
data_about.loc[index_id,'data_reference'] = 'l'
data_about.loc[index_id,'data_comment'] = 'Experiment L'
data_about.loc[index_id,'x_units'] = 'GeV'
data_about.loc[index_id,'y_units'] =  'cm^2'
data_about.loc[index_id,'x_rescale'] = '1'
data_about.loc[index_id,'y_rescale'] = '1'
data_about.loc[index_id,'year'] = '2023'
data_about.loc[index_id,'experiment'] = 'Experiment L'
data_about.loc[index_id,'spin_dependency'] = 'SD' # SI
data_about.loc[index_id,'result_type'] = 'Th' ## Proj, Exp
data_about.loc[index_id,'official'] = '1' ## 0
data_about.loc[index_id,'open'] = '1' ## 0
data_about.loc[index_id,'greatest_hit'] = '0' ## 1
data_about.loc[index_id,'public'] = '1' ## 0
data_about.loc[index_id,'created_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'updated_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'archived_at'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'creator_id'] = 0
data_about.loc[index_id,'rating'] = 0
data_about.loc[index_id,'date_of_announcement'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_official'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_start'] = '2024-07-08 11:07:00'
data_about.loc[index_id,'date_of_run_end'] = '2024-07-08 11:07:00'

data_about.loc[index_id,'default_color'] = 'black' ## 0
data_about.loc[index_id,'default_style'] = 'dotted' ## 0